<center>
    <h1 style="font-size: 12; font-weight: 900; ">--- 5 ---<h1>
    <h2>CSiAPI - ETABS y SAP2000</h2>
    <!-- <p><strong></strong</p> -->
</center>

In [16]:
import os, sys

import comtypes.client

import ctypes

## 0. Funciones.

Funciones utiles de limpiesa y restamblecimiento

In [17]:
# Elimina objetos existentes FrameObj
def delete_all_frames(smodel):
    # Get the names of all frame objects
    ret, all_frames, ret = smodel.FrameObj.GetNameList()

    # Delete each frame object
    for frame in all_frames:
        smodel.FrameObj.Delete(frame)

    print("All frames deleted successfully")
    
# Generacion e la geometria de la estructura
def create_structural_system(smodel, bay_width=10, story_height=10, num_bays=2, num_stories=2):
    # Delete existing frames
    delete_all_frames(smodel)
    base_points = []

    # Define material and section properties
    smodel.PropMaterial.SetMaterial("Concrete", 2)
    smodel.PropFrame.SetRectangle("Beam", "Concrete", 0.3, 0.5)
    smodel.PropFrame.SetRectangle("Column", "Concrete", 0.5, 0.5)

    # Create columns and beams
    for i in range(num_bays + 1):
        for j in range(num_stories):
            x_coord = i * bay_width
            z_coord = j * story_height
            smodel.FrameObj.AddByCoord(x_coord, 0, z_coord, x_coord, 0, z_coord + story_height, "", "Column", f"C{i}{j}")
            if j == 0:
                base_points.append(f"{x_coord}_0_{z_coord}")

    # Create beams
    for j in range(1, num_stories + 1):
        z_coord = j * story_height
        for i in range(num_bays):
            x_start = i * bay_width
            x_end = (i + 1) * bay_width
            smodel.FrameObj.AddByCoord(x_start, 0, z_coord, x_end, 0, z_coord, "", "Beam", f"B{j}{i}")

    print("Structural system created successfully")
    return base_points

Analisis y solicitud de resultados y Desbloque del modelo para implementar modificaciones

In [18]:
def run_analysis_and_get_joint_reactions(smodel, base_points):
    smodel.Analyze.RunAnalysis()
    smodel.Results.Setup.DeselectAllCasesAndCombosForOutput()
    smodel.Results.Setup.SetCaseSelectedForOutput("Dead")
    smodel.Results.Setup.SetCaseSelectedForOutput("Live")

    for each in base_points:
        print(smodel.Results.JointReact(each, 0))

def unlocked_model(smodel):
    # Unlock the model before running the analysis
    smodel.SetModelIsLocked(False)

## 1. API-CSI

In [19]:
# Unidades Base mks
m = 1
kg = 1
s = 1

# Otras Unidades
cm = 0.01*m
kgf = 9.80665*kg*m/s**2
tonf = 1000*kgf

# Constantes Físicas
g = 9.80665*m/s**2

# configuraciones de unidades
KN_m_C = 6
kgf_m_C = 8
Tonf_m_C = 12

In [20]:
connect_software = 2 # 1 para ETABS, 2 para SAP2000


In [21]:
if connect_software == 1:
    software = "SAP2000"
    conn = { 'app_adjunto': "CSI.SAP2000.API.SapObject", 'app_helper': 'SAP2000v1.Helper' }
elif (connect_software == 2):
    software = "ETABS"
    conn = { 'app_adjunto': "CSI.ETABS.API.ETABSObject", 'app_helper': "ETABSv1.Helper" }
else: 
    print("No tenemos soporte aun")
    sys.exit()

helper = comtypes.client.CreateObject(conn['app_helper'])
if connect_software == 1: helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)
if connect_software == 2: helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper);
try: Sap2000Object = helper.CreateObjectProgID(conn["app_adjunto"])
except (OSError, comtypes.COMError): print(f"Cannot start a new instance of the program({software}).")

Sap2000Object.ApplicationStart()

smodel = Sap2000Object.SapModel

In [22]:
# Initialize the model
smodel.InitializeNewModel()

smodel.File.NewBlank()

0

In [23]:
smodel.SetPresentUnits(KN_m_C)

0

In [24]:
# Creacion de materiales y geometria | Create the structural system
base_points = create_structural_system(
    smodel, 
    bay_width=4, 
    story_height=8, 
    num_bays=4, 
    num_stories=4
)

# carga en portico | apply loads

# Condiciones de borde o apoyos | restraints

smodel.View.RefreshView(
    0, 
    False
    );

All frames deleted successfully
Structural system created successfully


In [ ]:
# Save the model before analysis

# Se definen registros de salida
projectPath = r'C:\\CSi_API_Example';
if not os.path.exists(projectPath):
    try: os.makedirs(projectPath)
    except OSError: pass

ModelPath = projectPath + os.sep
# save model | guardar nuestro modelo
if connect_software == 1: ret = smodel.File.Save(ModelPath+'Frame-e2.sdb')
if connect_software == 2: ret = smodel.File.Save(ModelPath+'Frame-e2.edb', True)

## 2. Resultados

In [ ]:
# Run the analysis and get joint reactions
run_analysis_and_get_joint_reactions(smodel, base_points)

# Unlock and run the analysis again (if needed)
unlocked_model(smodel)
run_analysis_and_get_joint_reactions(smodel, base_points)

All frames deleted successfully
Structural system created successfully
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (), (), (), (), (), (), (), (), (), (), 1]
[0, (), (),

## 3. Cerrar la conneccion

In [25]:
if (smodel.ApplicationExit(False)): print("SAP2000, No se pudo cerrar")
else: print("Se cerro con exito.")

smodel = None
del smodel

Se cerro con exito.
